# 01 - Data: Spurious Environments

This notebook implements and visualizes the CIFAR-10 environments with spurious colored patches.

## Environments:
- **Env A (Spurious Aligned)**: Patch color matches label with probability 0.95
- **Env B (Spurious Flipped)**: Patch color matches label with probability 0.05
- **No Patch (OOD)**: Clean CIFAR-10 without any patches

## What this notebook does:
1. Creates all environment datasets
2. Visualizes sample images from each environment
3. Verifies alignment rates
4. Saves visualization figures to results/figures/

In [ ]:
import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Load configuration and set seeds
from src.config import get_config, set_seed, get_device, FIGURES_DIR, CIFAR10_CLASSES

config = get_config()
set_seed(config['seeds']['global'])
device = get_device()

print(f"Device: {device}")
print(f"Figures will be saved to: {FIGURES_DIR}")

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from src.data import (
    create_env_a_dataset,
    create_env_b_dataset,
    create_no_patch_dataset,
    create_mixed_env_dataset,
    denormalize,
    get_sample_batch,
)
from src.plotting import plot_sample_grid, save_figure

## 1. Create Datasets

In [ ]:
# Create all environment datasets
print("Creating datasets (this may download CIFAR-10)...\n")

# Training sets
env_a_train = create_env_a_dataset(train=True, config=config)
env_b_train = create_env_b_dataset(train=True, config=config)
mixed_train = create_mixed_env_dataset(env_a_fraction=0.5, train=True, config=config)

# Test sets
env_a_test = create_env_a_dataset(train=False, config=config)
env_b_test = create_env_b_dataset(train=False, config=config)
no_patch_test = create_no_patch_dataset(train=False, config=config)

print("Dataset sizes:")
print(f"  Env A train: {len(env_a_train)}")
print(f"  Env B train: {len(env_b_train)}")
print(f"  Mixed train: {len(mixed_train)}")
print(f"  Env A test (ID): {len(env_a_test)}")
print(f"  Env B test: {len(env_b_test)}")
print(f"  No patch test (OOD): {len(no_patch_test)}")

## 2. Verify Alignment Rates

In [ ]:
# Check alignment rates
print("Alignment rate verification:")
print("=" * 50)

env_a_rate = env_a_train.get_alignment_rate()
env_b_rate = env_b_train.get_alignment_rate()

print(f"\nEnv A (spurious aligned):")
print(f"  Expected alignment: {config['patch']['p_align_env_a']}")
print(f"  Actual alignment:   {env_a_rate:.4f}")
print(f"  Difference:         {abs(env_a_rate - config['patch']['p_align_env_a']):.4f}")

print(f"\nEnv B (spurious flipped):")
print(f"  Expected alignment: {config['patch']['p_align_env_b']}")
print(f"  Actual alignment:   {env_b_rate:.4f}")
print(f"  Difference:         {abs(env_b_rate - config['patch']['p_align_env_b']):.4f}")

# Sanity check
assert abs(env_a_rate - config['patch']['p_align_env_a']) < 0.02, "Env A alignment rate too far from expected!"
assert abs(env_b_rate - config['patch']['p_align_env_b']) < 0.02, "Env B alignment rate too far from expected!"
print("\nSanity check PASSED!")

## 3. Visualize Patch Colors

In [ ]:
# Show the color palette for each class
fig, ax = plt.subplots(figsize=(12, 3))

class_colors = config['patch']['class_colors']
n_classes = len(class_colors)

for i, (color, class_name) in enumerate(zip(class_colors, CIFAR10_CLASSES)):
    # Normalize color to [0, 1]
    norm_color = tuple(c / 255.0 for c in color)
    rect = plt.Rectangle((i, 0), 1, 1, facecolor=norm_color, edgecolor='black')
    ax.add_patch(rect)
    ax.text(i + 0.5, -0.2, class_name, ha='center', va='top', fontsize=10, rotation=45)
    ax.text(i + 0.5, 0.5, f"{color}", ha='center', va='center', fontsize=8,
            color='white' if sum(color) < 400 else 'black')

ax.set_xlim(0, n_classes)
ax.set_ylim(-0.5, 1)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Patch Color Palette by Class', fontsize=14, pad=20)

plt.tight_layout()
save_figure(fig, 'color_palette')
plt.show()

## 4. Visualize Environment A (Spurious Aligned)

In [ ]:
# Get sample batch from Env A
images_a, labels_a = get_sample_batch(env_a_train, n_samples=16, seed=42)

# Denormalize images for visualization
images_a_vis = np.array([denormalize(img, config) for img in images_a])

fig = plot_sample_grid(
    images_a_vis,
    labels_a.numpy(),
    title="Environment A (Spurious Aligned, p=0.95)",
    nrow=4,
    ncol=4,
    save_name='env_a_samples'
)
plt.show()

In [ ]:
# Verify: Check that patch colors mostly match labels
print("Env A: Verifying patch-label alignment...\n")

n_check = 100
aligned = 0
for i in range(n_check):
    _, label = env_a_train.cifar[i]
    patch_color = env_a_train.get_patch_color(i)
    expected_color = class_colors[label]
    if patch_color == expected_color:
        aligned += 1

print(f"First {n_check} samples: {aligned}/{n_check} aligned ({aligned/n_check*100:.1f}%)")

## 5. Visualize Environment B (Spurious Flipped)

In [ ]:
# Get sample batch from Env B
images_b, labels_b = get_sample_batch(env_b_train, n_samples=16, seed=42)

# Denormalize images for visualization
images_b_vis = np.array([denormalize(img, config) for img in images_b])

fig = plot_sample_grid(
    images_b_vis,
    labels_b.numpy(),
    title="Environment B (Spurious Flipped, p=0.05)",
    nrow=4,
    ncol=4,
    save_name='env_b_samples'
)
plt.show()

In [ ]:
# Verify: Check that patch colors mostly DON'T match labels
print("Env B: Verifying patch-label misalignment...\n")

n_check = 100
aligned = 0
for i in range(n_check):
    _, label = env_b_train.cifar[i]
    patch_color = env_b_train.get_patch_color(i)
    expected_color = class_colors[label]
    if patch_color == expected_color:
        aligned += 1

print(f"First {n_check} samples: {aligned}/{n_check} aligned ({aligned/n_check*100:.1f}%)")
print(f"Expected ~{config['patch']['p_align_env_b']*100}% alignment")

## 6. Visualize No-Patch (OOD) Test Set

In [ ]:
# Get sample batch from No Patch dataset
images_np, labels_np = get_sample_batch(no_patch_test, n_samples=16, seed=42)

# Denormalize images for visualization
images_np_vis = np.array([denormalize(img, config) for img in images_np])

fig = plot_sample_grid(
    images_np_vis,
    labels_np.numpy(),
    title="No Patch (OOD Test Set)",
    nrow=4,
    ncol=4,
    save_name='no_patch_samples'
)
plt.show()

## 7. Side-by-Side Comparison

In [ ]:
# Create side-by-side comparison figure
fig, axes = plt.subplots(3, 8, figsize=(16, 6))

# Get samples with same seed for comparison
images_a, labels_a = get_sample_batch(env_a_train, n_samples=8, seed=123)
images_b, labels_b = get_sample_batch(env_b_train, n_samples=8, seed=123)
images_np, labels_np = get_sample_batch(no_patch_test, n_samples=8, seed=123)

# Note: these are different images due to different datasets
# But we use the same seed for consistent sampling within each dataset

row_titles = ['Env A (aligned)', 'Env B (flipped)', 'No Patch (OOD)']
all_images = [images_a, images_b, images_np]
all_labels = [labels_a, labels_b, labels_np]

for row, (images, labels, title) in enumerate(zip(all_images, all_labels, row_titles)):
    images_vis = [denormalize(img, config) for img in images]
    for col, (img, label) in enumerate(zip(images_vis, labels)):
        axes[row, col].imshow(img)
        axes[row, col].axis('off')
        if row == 0:
            axes[row, col].set_title(CIFAR10_CLASSES[label], fontsize=9)
    axes[row, 0].set_ylabel(title, fontsize=10, rotation=90, labelpad=40)

plt.suptitle('Environment Comparison', fontsize=14, y=1.02)
plt.tight_layout()
save_figure(fig, 'environment_comparison')
plt.show()

## 8. Class Distribution Check

In [ ]:
# Verify class distribution is balanced
from collections import Counter

def get_class_distribution(dataset):
    labels = [dataset.cifar[i][1] for i in range(len(dataset))]
    return Counter(labels)

env_a_dist = get_class_distribution(env_a_train)

fig, ax = plt.subplots(figsize=(10, 5))

classes = list(range(10))
counts = [env_a_dist[c] for c in classes]

bars = ax.bar(classes, counts, color='steelblue', edgecolor='black')
ax.set_xticks(classes)
ax.set_xticklabels(CIFAR10_CLASSES, rotation=45, ha='right')
ax.set_xlabel('Class')
ax.set_ylabel('Count')
ax.set_title('CIFAR-10 Class Distribution (Training Set)')

# Add count labels on bars
for bar, count in zip(bars, counts):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
            str(count), ha='center', va='bottom', fontsize=9)

plt.tight_layout()
save_figure(fig, 'class_distribution')
plt.show()

print(f"\nTotal samples: {sum(counts)}")
print(f"Expected per class: {sum(counts)//10}")

## 9. Summary

In [ ]:
print("=" * 60)
print("DATA PREPARATION COMPLETE")
print("=" * 60)
print(f"""
Environments created:

1. Environment A (Spurious Aligned)
   - Patch color matches true label with p={config['patch']['p_align_env_a']}
   - Training: {len(env_a_train)} samples
   - Test (ID): {len(env_a_test)} samples
   - Actual alignment rate: {env_a_rate:.4f}

2. Environment B (Spurious Flipped)
   - Patch color matches true label with p={config['patch']['p_align_env_b']}
   - Training: {len(env_b_train)} samples
   - Actual alignment rate: {env_b_rate:.4f}

3. No Patch (OOD Test)
   - Clean CIFAR-10 without patches
   - Test: {len(no_patch_test)} samples

4. Mixed Environment (for robust training)
   - 50% Env A + 50% Env B
   - Training: {len(mixed_train)} samples

Patch configuration:
   - Size: {config['patch']['size']}x{config['patch']['size']} pixels
   - Position: {config['patch']['position']}
   - Colors: 10 distinct colors (one per class)

Figures saved:
   - {FIGURES_DIR / 'color_palette.png'}
   - {FIGURES_DIR / 'env_a_samples.png'}
   - {FIGURES_DIR / 'env_b_samples.png'}
   - {FIGURES_DIR / 'no_patch_samples.png'}
   - {FIGURES_DIR / 'environment_comparison.png'}
   - {FIGURES_DIR / 'class_distribution.png'}

Next: Run 02_train_models.ipynb to train the 4 models.
""")